In [40]:
import pandas as pd
import os
import datashader, hvplot, holoviews as hv 
import panel as pn
import hvplot.pandas
pn.extension(throttled=True)
hv.extension('bokeh', 'matplotlib')

### Open datasets

In [3]:
dir_data = 'data'
# Import Kepler LC for a single RR Lyr object
cat_kepler = pd.read_csv(os.path.join(dir_data,'kepler_RRLyr.csv')) 
# Import LSST multi-band LCs for a number of RR Lyr candidates
cat_lsst = pd.read_pickle(os.path.join(dir_data,'lsst_RRLyr.pkl'))

In [4]:
bands_lsst = 'ugrizy'
colors = ['#cc78a7','#0072b2','#009e73',
          '#f0e442','#e69f00','#d55e00']

#### Inspect the data

In [5]:
cat_kepler.head()

,time,flux,flux_err,quality,timecorr,centroid_col,centroid_row,cadenceno,sap_flux,sap_flux_err,...,psf_centr1,psf_centr1_err,psf_centr2,psf_centr2_err,mom_centr1,mom_centr1_err,mom_centr2,mom_centr2_err,pos_corr1,pos_corr2
0,131.512404,10180609.0,78.926155,128,0.001410,653.372470,51.053028,1105,10129629.0,79.186980,...,NaN,NaN,NaN,NaN,653.372470,0.000006,51.053028,0.000063,0.011782,-0.010195
1,131.532839,10013518.0,78.233770,128,0.001411,653.372292,51.053872,1106,9949931.0,78.459984,...,NaN,NaN,NaN,NaN,653.372292,0.000006,51.053872,0.000062,0.011726,-0.010246
2,131.553273,9852474.0,77.673160,128,0.001412,653.372167,51.044559,1107,9783633.0,77.827200,...,NaN,NaN,NaN,NaN,653.372167,0.000006,51.044559,0.000062,0.011575,-0.010089
3,131.573707,9722936.0,77.109710,128,0.001413,653.371408,51.045081,1108,9651452.0,77.313590,...,NaN,NaN,NaN,NaN,653.371408,0.000006,51.045081,0.000062,0.011366,-0.009939
4,131.594142,9717073.0,77.103550,0,0.001414,653.372167,51.052828,1109,9646289.0,77.262634,...,NaN,NaN,NaN,NaN,653.372167,0.000006,51.052828,0.000062,0.011526,-0.010702


In [6]:
cat_kepler.columns

Index(['time', 'flux', 'flux_err', 'quality', 'timecorr', 'centroid_col',
       'centroid_row', 'cadenceno', 'sap_flux', 'sap_flux_err', 'sap_bkg',
       'sap_bkg_err', 'pdcsap_flux', 'pdcsap_flux_err', 'sap_quality',
       'psf_centr1', 'psf_centr1_err', 'psf_centr2', 'psf_centr2_err',
       'mom_centr1', 'mom_centr1_err', 'mom_centr2', 'mom_centr2_err',
       'pos_corr1', 'pos_corr2'],
      dtype='object')

In [7]:
cat_lsst.head()

,band,ccdVisitId,coord_ra,coord_dec,objectId,psfFlux,psfFluxErr,psfMag,ccdVisitId2,band2,expMidptMJD,zeroPoint
0,y,1032263018,62.462569,-44.11336,1251384969897480052,-515.183603,1697.218490,NaN,1032263018,y,61100.069706,30.602301
1,y,1033987172,62.462569,-44.11336,1251384969897480052,3151.738459,1686.955775,22.653625,1033987172,y,61102.068464,30.606100
2,u,675163080,62.462569,-44.11336,1251384969897480052,183.449123,209.242045,25.741211,675163080,u,60582.247144,30.469101
3,y,443055067,62.462569,-44.11336,1251384969897480052,-704.848327,1624.400086,NaN,443055067,y,60215.203585,30.612801
4,u,466722002,62.462569,-44.11336,1251384969897480052,382.472233,278.926670,24.943500,466722002,u,60261.078221,30.461201


In [8]:
cat_lsst.columns

Index(['band', 'ccdVisitId', 'coord_ra', 'coord_dec', 'objectId', 'psfFlux',
       'psfFluxErr', 'psfMag', 'ccdVisitId2', 'band2', 'expMidptMJD',
       'zeroPoint'],
      dtype='object')

In [19]:
lc = cat_lsst[cat_lsst['objectId'] == cat_lsst['objectId'].unique()[7]]

### Multiple folded LCs on a single plot

In [20]:
from astropy.timeseries import LombScargle
import astropy.units as u
import numpy as np

In [21]:
min_period = 0.2 * u.day
max_period = 1 * u.day

min_freq_search = 1.0 / max_period
max_freq_search = 1.0 / min_period

In [22]:
mjd_days = {}
mags = {}
for b in bands_lsst:
    mjd_days[b] = np.array(lc[lc['band']==b]["expMidptMJD"]) * u.day
    mags[b] = np.array(lc[lc['band']==b]["psfMag"])

In [31]:
#frequency = {}
#power = {}
freq = {}
for b in bands_lsst:
    freq[b] = pd.DataFrame()
    freq[b]['freq'], freq[b]['power'] = LombScargle(mjd_days[b], mags[b]).autopower(
        minimum_frequency=min_freq_search, maximum_frequency=max_freq_search
    )

In [38]:
all_peak_freqs = []

for b in bands_lsst:
    # find the index with maximum power (= peakbin)
    peakbin = np.argmax(freq[b]['power'])

    # Store the frequency corresponding to the peak power in each band (band)
    all_peak_freqs.append(freq[b]['freq'][peakbin].value)

# Convert the frequencies from a list to and array:
all_peak_freqs = np.array(all_peak_freqs)

# Calculate the mean of the "best-fit" frequencies:
mean_peak_freq = np.mean(all_peak_freqs)

print("Mean frequency:", mean_peak_freq)
print("Mean period:", 1.0 / mean_peak_freq, " days")
print("\nugrizy frequency results:\n", all_peak_freqs)

Mean frequency: 2.178158712764198
Mean period: 0.45910336750940767  days

ugrizy frequency results:
 [2.17807944 2.17816158 2.17813256 2.17819998 2.178213   2.17816572]


In [42]:
freq['r']

,freq,power
0,1.000000,0.027165
1,1.000113,0.026305
2,1.000226,0.033144
3,1.000339,0.042981
4,1.000452,0.053250
...,...,...
35398,4.999572,0.010290
35399,4.999685,0.015327
35400,4.999798,0.021651
35401,4.999911,0.016651


In [75]:
freq['r'].hvplot(x='freq',y='power').opts(width=300,height=400,xlim=(freq['r']['freq'].min().value,freq['r']['freq'].max().value))

:Curve   [freq]   (power)

In [63]:
best_period = 1.0 / mean_peak_freq

In [71]:
phased = {}

# Number of elapsed periods since the first measurement:
t0 = np.min(mjd_days["r"].value)

for band in bands_lsst:
    phased[band] = pd.DataFrame()
    phased[band]['mjd_norm'] = (mjd_days[band].value - t0) / best_period
    phased[band]['phase'] = np.mod(phased[band]['mjd_norm'], 1.0)
    phased[band]['mags'] = mags[band]
    phased[band]['mags_norm'] = mags[band] - np.mean(mags[band])

In [98]:
phased['r'].hvplot.scatter(x='phase',y='mags_norm')

:Scatter   [phase]   (mags_norm)

### Linked plots for multi-band LC

Using interlinked plots, we can synchronise inspection of the same regions of the parameter space
on different plots. E.g. on the plots below we can zoom in a certain part of the time range and look at the
observations within this time range across all photometric bands.

In [76]:
# Creating separate scatter plots for every photometric band
plots_lsst = {}
for i,b in enumerate(bands_lsst):
    plots_lsst[b] = phased[b].hvplot.scatter(x='phase',y='mags_norm',title=b, color=colors[i]).opts(width=500,height=300)

In [77]:
# Creating a linked layout
ls = hv.link_selections.instance()

In [78]:
# Displaying the plots in a vertical column
(ls(plots_lsst['u'] + 
    plots_lsst['g'] + 
    plots_lsst['r'] + 
    plots_lsst['i'] + 
    plots_lsst['z'] + 
    plots_lsst['y'])).cols(2)

:Layout
   .Scatter.I   :DynamicMap   []
      :Overlay
         .Scatter.I   :Scatter   [phase]   (mags_norm)
         .NdOverlay.I :NdOverlay   [Element]
            :VSpan   [x,y]
         .Scatter.II  :Scatter   [phase]   (mags_norm)
   .Scatter.II  :DynamicMap   []
      :Overlay
         .Scatter.I   :Scatter   [phase]   (mags_norm)
         .NdOverlay.I :NdOverlay   [Element]
            :VSpan   [x,y]
         .Scatter.II  :Scatter   [phase]   (mags_norm)
   .Scatter.III :DynamicMap   []
      :Overlay
         .Scatter.I   :Scatter   [phase]   (mags_norm)
         .NdOverlay.I :NdOverlay   [Element]
            :VSpan   [x,y]
         .Scatter.II  :Scatter   [phase]   (mags_norm)
   .Scatter.IV  :DynamicMap   []
      :Overlay
         .Scatter.I   :Scatter   [phase]   (mags_norm)
         .NdOverlay.I :NdOverlay   [Element]
            :VSpan   [x,y]
         .Scatter.II  :Scatter   [phase]   (mags_norm)
   .Scatter.V   :DynamicMap   []
      :Overlay
         .Scatter.I   :Scatter   [phase]   (mags_norm)
         .NdOverlay.I :NdOverlay   [Element]
            :VSpan   [x,y]
         .Scatter.II  :Scatter   [phase]   (mags_norm)
   .Scatter.VI  :DynamicMap   []
      :Overlay
         .Scatter.I   :Scatter   [phase]   (mags_norm)
         .NdOverlay.I :NdOverlay   [Element]
            :VSpan   [x,y]
         .Scatter.II  :Scatter   [phase]   (mags_norm)

In [81]:
phased_all = pd.DataFrame()
for b in bands_lsst:
    phased[b]['band'] = b
    phased_all = pd.concat((phased_all,phased[b]))

In [101]:
phased_all.sort_values('phase').hvplot.line(x='phase',y='mags_norm', by='band', hover=True,
                  xlabel='Time (ms)', ylabel='Amplitude (µV)', min_height=300,
                  title="Datashade Multiple Lines Per Category",line_width=3)

:NdOverlay   [band]
   :Curve   [phase]   (mags_norm)